In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('../input/gtzan-dataset-music-genre-classification/Data/features_30_sec.csv')

In [ ]:
def get_image_path(x):
    splited = x.split('.')
    folder = '../input/gtzan-dataset-music-genre-classification/Data/images_original/'+splited[0] + '/'
    filename = splited[0]+splited[1]+'.png'
    return folder+filename

In [ ]:
data['path'] = data['filename'].apply(get_image_path)
data

In [ ]:
data = data[['filename','path']]

In [ ]:
data['genre'] = data['filename'].apply(lambda x:x.split('.')[0])
data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(
         data,  test_size=0.2, random_state=15)

In [ ]:
len(X_train.genre.unique()),len(X_test.genre.unique())

In [ ]:
X_train.genre.hist()

In [ ]:
X_test.genre.hist()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator()

In [ ]:
it = datagen.flow_from_dataframe(
    data,
    x_col="path",
    y_col="genre",batch_size=1)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(next(it)[0][0])

In [ ]:
next(it)[0][0].shape

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense,Input
inp = Input(shape = next(it)[0][0].shape)
model = ResNet50(
    include_top=False,
    weights="imagenet",
    
    classes=10,
    input_tensor = inp
)

In [ ]:
for layer in model.layers:
    layer.trainable=False

In [ ]:
model.input.shape

In [ ]:
from tensorflow.keras.layers import BatchNormalization,Dropout,Conv2D
new_model = Sequential()
new_model.add(model)
new_model.add(Flatten())
# new_model.add(BatchNormalization())
# new_model.add(Dense(256, activation='relu'))
# new_model.add(Dropout(0.5))
new_model.add(BatchNormalization())
new_model.add(Dense(128, activation='relu'))
new_model.add(Dropout(0.5))
new_model.add(BatchNormalization())
new_model.add(Dense(64, activation='relu'))
new_model.add(Dropout(0.5))
new_model.add(Dense(10, activation='softmax'))

In [ ]:
new_model.summary()

In [ ]:
new_model.compile(loss = 'categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(
    monitor='val_accuracy', restore_best_weights=True, patience=3
)

new_model.fit(datagen.flow_from_dataframe(
    X_train,
    x_col="path",
    y_col="genre",batch_size=100),
              validation_data=datagen.flow_from_dataframe(
            X_test,
            x_col="path",
            y_col="genre",batch_size=64),epochs=50,callbacks=[es])

In [ ]:
new_model.evaluate(datagen.flow_from_dataframe(
    X_test,
    x_col="path",
    y_col="genre",batch_size=64))